In [10]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import zipfile
from collections import defaultdict
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

base_path = f'/data/{origin}et/Auxiliary/landcover/'
LCs = ['Forest_type', 'Crop_types', 'Grassland', 'Impervious', 'Water_and_wetness']
nodata_vals = [255, 65535, 255, 255, 255]

lc_uniq = {'Crop_types' : [0, 1110, 1120, 1130, 1140, 1150, 1210, 1220, 1310, 1320, 1410, 1420, 1430, 1440, 2100, 2200, 2310, 2320, 3100, 3200], 
            'Forest_type' : [0, 1, 2],
            'Grassland' : [0, 1],
            'Impervious' : [0, 1],
            'Water_and_wetness' : [0, 1, 2, 3, 4, 253, 254]
            }

lc_codes = {'Crop_types' : ['No_cropland', 'Wheat', 'Barley', 'Maize', 'Rice', 'Other_Cereals', 'Fresh_Vegetables', 'Dry_Pulses',
                            'Potatoes', 'Sugar_Beet', 'Sunflower', 'Soybeans', 'Rapeseed', 'Flax_cotton_hemp', 'Grapes',
                            'Olives', 'Fruits', 'Nuts', 'Undecided_arable_crop', 'Undecided_perrenial_crop'], 
            'Forest_type' : ['No_forest', 'Broadleaved_forest', 'Coniferous_forest'],
            'Grassland' : ['No_grassland', 'Grassland'],
            'Impervious' : ['No_impervious', 'Impervious'],
            'Water_and_wetness' : ['Dry', 'Permanent_water', 'Temporary_water',  'Permanent_wet', 'Temporary_wet', 'Sea_water', 
                                    'unclassifiable']}

def count_known_vals_bincount(slice, known_vals):
    counts = np.bincount(slice, minlength=np.max(known_vals) + 1)
    return dict(zip([str(known_val) for known_val in known_vals], [int(c) for c in counts[known_vals]])) # counts[known_vals]

In [ ]:
# extract downloaded landcover files
for fold in ['Forest_type', 'Crop_types', 'Grassland']:
    outPath = f'{base_path}extracted/{fold}'
    if os.path.isdir(outPath):
        continue
    else:
        os.makedirs(outPath)
        for file in getFilelist(f'{base_path}raw/{fold}', '.zip'):
            with zipfile.ZipFile(file, 'r') as zip:
                zip.extractall(outPath)

In [ ]:
# make vrts

for i, LC in enumerate(LCs):
    files = getFilelist(f'{base_path}extracted/{LC}', 'tif')
    vrt = gdal.BuildVRT(f'{base_path}extracted/{LC}/{LC}.vrt', files, separate = False, options=gdal.BuildVRTOptions(VRTNodata=nodata_vals[i]))
    vrt = None
    

In [ ]:
####### press landcover into S3
# when warping the Lc raster, their spatial resolution should be kept while aligning them with the mask (S3 pixels). As a trade-off, the pixel_size from the 
# landcovermaps (reprojected to the mask's crs) will be divided by the pixel_size from mask, and then rounded (floor)
xsize, ysize = get_pixel_size_in_target_crs(getFilelist(f'{base_path}extracted/{LCs[0]}', '.vrt')[0],'/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif')
mask = gdal.Open('/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif')
gt = mask.GetGeoTransform()
xdiv = math.floor(gt[1] / xsize)
ydiv = math.floor(-gt[5] / ysize)

if xdiv == ydiv:
    for LC in LCs:
        files = getFilelist(f'{base_path}extracted/{LC}', '.vrt')
        if os.path.exists(f'{base_path}warped/{LC}.tif'):
            continue
        else:
            for file in files:
                warp_raster_to_reference(file, '/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif', f'{base_path}warped/{LC}.tif', 'nearest', xdiv) 
# mask them
if os.path.exists( f'/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask{xdiv}.tif'):
    pass
else:
    warp_raster_to_reference('/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif', 
                            f'{base_path}warped/{LCs[0]}.tif', 
                            f'/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask{xdiv}.tif', 'nearest')


In [ ]:
# mask em
warped = getFilelist(f'{base_path}warped/', '.tif')
for warp in warped:
    if 'Crop_types' in warp:
        nodata_val = 65535
    else:
        nodata_val = 255
    print(warp)
    print(nodata_val)
    mask_raster(warp, f'/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask{xdiv}.tif', warp.split('.tif')[0] + '_masked.tif', nodata_val)
    os.remove(file)

In [ ]:
# get warped and masked tifs
S3 = gdal.Open(f'/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif')
S3_arr = S3.GetRasterBand(1).ReadAsArray()
start_ind_row = [i for i in range(0, (S3_arr.shape[0] * xdiv) + 1, xdiv)]
end_ind_row = start_ind_row[1:]
start_ind_col = [i for i in range(0, (S3_arr.shape[1] * xdiv) + 1, xdiv)]
end_ind_col = start_ind_col[1:]

warpedM = getFilelist(f'{base_path}warped/', '.tif')

In [13]:
for LC in LCs:
    # read-in Lc raster
    ds = gdal.Open([warp for warp in warpedM if LC in warp][0], 0)
    arr = ds.GetRasterBand(1).ReadAsArray()
    conti = []
    for row in range(len(end_ind_row)):
        for col in range(len(end_ind_col)):
            # print(f'row: {row}\ncol: {col}\nstart_ind_row: {start_ind[row]}\nend_ind_row: {end_ind[row]}\nstart_ind_col: {start_ind[col]}\nend_ind_col: {end_ind[col]}')
            # conti.append(count_known_vals_bincount(arr[start_ind_row[row]:end_ind_row[row], start_ind_col[col]:end_ind_col[col]].flatten(), lc_uniq[LC]))
            res = count_known_vals_bincount(arr[start_ind_row[row]:end_ind_row[row], start_ind_col[col]:end_ind_col[col]].flatten(), lc_uniq[LC])
            res = dict(zip(lc_codes[LC], list(res.values())))
            res['LC'] = LC
            res['row_col'] = f'{row}_{col}'
            conti.append(res)

    merged = defaultdict(list)
    for d in conti:
        for k, v in d.items():
            merged[k].append(v)

    df = pd.DataFrame(merged)
    df.to_csv(f'/data/{origin}et/Auxiliary/landcover/csv/extract_{LC}.csv', index=False)